<a href="https://colab.research.google.com/github/AIFFEL-GN-2nd/TotochTeam1/blob/main/day_4/day4_%EC%8B%A4%EC%8A%B5_%5B%EC%9D%B4%EB%A6%84%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이웃집 토토치 파이토치 : Day 4

📢 해당 게시물은 파이토치 공식 튜토리얼 중 
[컴퓨터 비전(VISION)을 위한 전이학습](https://tutorials.pytorch.kr/beginner/transfer_learning_tutorial.html)를 읽고 직접 작성해보는 실습 노트북입니다.  

In [ ]:
from IPython.display import Image
Image("./img/image.png")

🔎 Transfer learning은 무엇이고 pre-trained과 다른점이 무엇일까요?   
👉 (괄호를 지우고 적어주세요!)  

In [ ]:
Image("./img/gpu.png")

시작하기 전에 런타임을 GPU로 바꿔주세요!!

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import models, transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import copy

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

plt.ion()   # 대화형 모드

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCH = 10
BATCH_SIZE = 8
FC_LAYER_SIZE = 128
LR = 0.01
DROOUT = 0.5
OPTIMIZER = 'sgd'

## 1) Dataset 준비

Transfer Leaning을 준비하기 위해서는 (1)내가 원하는 Dataset 과 (2) pre-trained model 이 필요합니다.

In [ ]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
test_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_dataset = datasets.CIFAR10(root = '../CIFAR10', train = True, 
                               download = True, transform = train_transform) 

test_dataset = datasets.CIFAR10(root = '../CIFAR10', train = False,
                               download = True, transform = test_transform)

# Subset을 사용하면 Dataset의 부분 집합만 가져올 수 있음.
train_sub_dataset = Subset(train_dataset, indices=range(0, len(train_dataset), 5))
test_sub_dataset = Subset(test_dataset, indices=range(0, len(test_dataset), 5))

train_loader = DataLoader(dataset = train_sub_dataset, batch_size = BATCH_SIZE,
                         shuffle = True, num_workers=2)

test_loader = DataLoader(dataset = test_sub_dataset,
                         batch_size = BATCH_SIZE)

dataloaders = {
    'train':train_loader,
    'test':test_loader
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### A. 데이터 시각화

In [ ]:
def imshow(img):
    img = img / 2 + 0.5    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(BATCH_SIZE)))

## 2) Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

## 3) Fine-tuning 모델

In [ ]:
finetun_model = models.resnet18(pretrained=True)
num_ftrs = finetun_model.fc.in_features
finetun_model._fc = nn.Linear(num_ftrs, 2)

finetun_model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(finetun_model.parameters(), lr=LR, momentum=0.9)

## 4) 학습 진행하기

In [ ]:
model = finetun_model

train_acc_list =[]
train_loss_list = []
test_acc_list = []
test_loss_list = []

since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in tqdm(range(EPOCH)):
    print('Epoch {}/{}'.format(epoch, EPOCH - 1))
    print('-' * 20)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            # print(labels)
            # print(labels.shape)
            # break
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
        
        print('>> Phase: ', phase)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        if phase == 'train':
            train_acc_list.append(epoch_acc)
            train_loss_list.append(epoch_loss)
        else:
            test_acc_list.append(epoch_acc)
            test_loss_list.append(epoch_loss)
            
        # deep copy the model
        if phase == 'test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())


time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best test Acc: {:4f}'.format(best_acc))

# load best model weights
# model.load_state_dict(best_model_wts)

torch.save(best_model_wts , './pretrained_vgg_orignal_cifar10.pt')

## 5) 학습 결과 시각화

In [ ]:
plt.plot(train_loss_list)
plt.plot(test_loss_list)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(train_acc_list)
plt.plot(test_acc_list)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()